## Setup

In [1]:
import pandas as pd
import numpy as np
import os
from langchain.document_loaders import PyPDFLoader, UnstructuredPDFLoader, PyPDFium2Loader
from langchain.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

In [19]:
import os
import json
import requests

BASE_URL = os.environ.get('OLLAMA_HOST', 'http://localhost:11434')

# Generate a response for a given prompt with a provided model. This is a streaming endpoint, so will be a series of responses.
# The final response object will include statistics and additional data from the request. Use the callback function to override
# the default handler.
def generate(model_name, prompt, system=None, template=None, context=None, options=None, callback=None):
    try:
        url = f"{BASE_URL}/api/generate"
        payload = {
            "model": model_name, 
            "prompt": prompt, 
            "system": system, 
            "template": template, 
            "context": context, 
            "options": options
        }
        
        # Remove keys with None values
        payload = {k: v for k, v in payload.items() if v is not None}
        
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()
            
            # Creating a variable to hold the context history of the final chunk
            final_context = None
            
            # Variable to hold concatenated response strings if no callback is provided
            full_response = ""

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)
                    
                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # If this is not the last chunk, add the "response" field value to full_response and print it
                        if not chunk.get("done"):
                            response_piece = chunk.get("response", "")
                            full_response += response_piece
                            print(response_piece, end="", flush=True)
                    
                    # Check if it's the last chunk (done is true)
                    if chunk.get("done"):
                        final_context = chunk.get("context")
            
            # Return the full response and the final context
            return full_response, final_context
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None, None

# Create a model from a Modelfile. Use the callback function to override the default handler.
def create(model_name, model_path, callback=None):
    try:
        url = f"{BASE_URL}/api/create"
        payload = {"name": model_name, "path": model_path}
        
        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the status
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the status
                    chunk = json.loads(line)

                    if callback:
                        callback(chunk)
                    else:
                        print(f"Status: {chunk.get('status')}")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Pull a model from a the model registry. Cancelled pulls are resumed from where they left off, and multiple
# calls to will share the same download progress. Use the callback function to override the default handler.
def pull(model_name, insecure=False, callback=None):
    try:
        url = f"{BASE_URL}/api/pull"
        payload = {
            "name": model_name,
            "insecure": insecure
        }

        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)

                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # Print the status message directly to the console
                        print(chunk.get('status', ''), end='', flush=True)
                    
                    # If there's layer data, you might also want to print that (adjust as necessary)
                    if 'digest' in chunk:
                        print(f" - Digest: {chunk['digest']}", end='', flush=True)
                        print(f" - Total: {chunk['total']}", end='', flush=True)
                        print(f" - Completed: {chunk['completed']}", end='\n', flush=True)
                    else:
                        print()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# Push a model to the model registry. Use the callback function to override the default handler.
def push(model_name, insecure=False, callback=None):
    try:
        url = f"{BASE_URL}/api/push"
        payload = {
            "name": model_name,
            "insecure": insecure
        }

        # Making a POST request with the stream parameter set to True to handle streaming responses
        with requests.post(url, json=payload, stream=True) as response:
            response.raise_for_status()

            # Iterating over the response line by line and displaying the details
            for line in response.iter_lines():
                if line:
                    # Parsing each line (JSON chunk) and extracting the details
                    chunk = json.loads(line)

                    # If a callback function is provided, call it with the chunk
                    if callback:
                        callback(chunk)
                    else:
                        # Print the status message directly to the console
                        print(chunk.get('status', ''), end='', flush=True)
                    
                    # If there's layer data, you might also want to print that (adjust as necessary)
                    if 'digest' in chunk:
                        print(f" - Digest: {chunk['digest']}", end='', flush=True)
                        print(f" - Total: {chunk['total']}", end='', flush=True)
                        print(f" - Completed: {chunk['completed']}", end='\n', flush=True)
                    else:
                        print()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

# List models that are available locally.
def list():
    try:
        response = requests.get(f"{BASE_URL}/api/tags")
        response.raise_for_status()
        data = response.json()
        models = data.get('models', [])
        return models

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Copy a model. Creates a model with another name from an existing model.
def copy(source, destination):
    try:
        # Create the JSON payload
        payload = {
            "source": source,
            "destination": destination
        }
        
        response = requests.post(f"{BASE_URL}/api/copy", json=payload)
        response.raise_for_status()
        
        # If the request was successful, return a message indicating that the copy was successful
        return "Copy successful"

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Delete a model and its data.
def delete(model_name):
    try:
        url = f"{BASE_URL}/api/delete"
        payload = {"name": model_name}
        response = requests.delete(url, json=payload)
        response.raise_for_status()
        return "Delete successful"
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# Show info about a model.
def show(model_name):
    try:
        url = f"{BASE_URL}/api/show"
        payload = {"name": model_name}
        response = requests.post(url, json=payload)
        response.raise_for_status()
        
        # Parse the JSON response and return it
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def heartbeat():
    try:
        url = f"{BASE_URL}/"
        response = requests.head(url)
        response.raise_for_status()
        return "Ollama is running"
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return "Ollama is not running"

In [20]:
import uuid
import pandas as pd
import numpy as np
import sys

import json
# import ollama.client as client


def extractConcepts(prompt: str, metadata={}, model="mistral-openorca:latest"):
    SYS_PROMPT = (
        "Your task is extract the key concepts (and non personal entities) mentioned in the given context. "
        "Extract only the most important and atomistic concepts, if  needed break the concepts down to the simpler concepts."
        "Categorize the concepts in one of the following categories: "
        "[event, concept, place, object, document, organisation, condition, misc]\n"
        "Format your output as a list of json with the following format:\n"
        "[\n"
        "   {\n"
        '       "entity": The Concept,\n'
        '       "importance": The concontextual importance of the concept on a scale of 1 to 5 (5 being the highest),\n'
        '       "category": The Type of Concept,\n'
        "   }, \n"
        "{ }, \n"
        "]\n"
    )
    response, _ = generate(model_name=model, system=SYS_PROMPT, prompt=prompt)
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result


def graphPrompt(input: str, metadata={}, model="mistral-openorca:latest"):
    if model == None:
        model = "mistral-openorca:latest"

    # model_info = client.show(model_name=model)
    # print( chalk.blue(model_info))

    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    response, _ = generate(model_name=model, system=SYS_PROMPT, prompt=USER_PROMPT)
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


def df2ConceptsList(dataframe: pd.DataFrame) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: extractConcepts(
            row.text, {"chunk_id": row.chunk_id, "type": "concept"}
        ),
        axis=1,
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def concepts2Df(concepts_list) -> pd.DataFrame:
    ## Remove all NaN entities
    concepts_dataframe = pd.DataFrame(concepts_list).replace(" ", np.nan)
    concepts_dataframe = concepts_dataframe.dropna(subset=["entity"])
    concepts_dataframe["entity"] = concepts_dataframe["entity"].apply(
        lambda x: x.lower()
    )

    return concepts_dataframe


def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

## Load Documents

In [2]:
loader = DirectoryLoader('random_mrns/deid/', show_progress=True)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
print(pages[3].page_content)

100%|██████████| 8/8 [00:13<00:00,  1.73s/it]

Number of chunks =  4738
Subjective:       Patient ID: Jerry Adams is a 69 y.o. male. HPI  Patient is here for follow-up on chronic medical problems  He has prostate cancer he started radiation therapy this week, he reports that he is feeling down and he has been eating more for comfort. He also reports that he has good supportive system and he is trying to walk on daily basis. He has hypertension he is compliant with his medications he denies chest pain shortness of breath  He has hyperlipidemia tolerating statin    The following portions of the patient's history were reviewed and updated as appropriate: allergies, current medications, past family history, past medical history, past social history, past surgical history and problem list. Review of Systems   Constitutional: Negative. HENT: Negative for rhinorrhea, sinus pain and sore throat. Eyes: Negative. Respiratory: Negative. Cardiovascular: Negative. Gastrointestinal: Negative. Genitourinary: Negative. Musculoskeletal: Negative. N

## Create a dataframe of all the chunks

In [5]:
df = documents2Dataframe(pages)
print(df.shape)
df.head()

(4738, 3)


,text,source,chunk_id
0,Subjective: Patient ID: Jerry Adams is a...,random_mrns/deid/19064752_3.txt,38d3e5b0b5854945bd1ac139e7717c04
1,"Pupils: Pupils are equal, round, and reactive ...",random_mrns/deid/19064752_3.txt,b739c01a3f914a1c836f8890d6b913cb
2,Diagnoses and all orders for this visit: Es...,random_mrns/deid/19064752_3.txt,17668bec3a87441ca65973b17eb1f444
3,Subjective: Patient ID: Jerry Adams is a...,random_mrns/deid/19064752_3.txt,87e240c8608e4d428d064fb83e514772
4,"Pupils: Pupils are equal, round, and reactive ...",random_mrns/deid/19064752_3.txt,5eafdb7d105d4f9b903cb3fca7be9761


In [22]:
#select only the first 10 chunks
df = df.iloc[:10]
df.shape

(10, 3)

In [7]:
print("Word Count Statistics")
print(df.text.str.split().str.len().describe())

Word Count Statistics
count    4738.000000
mean      184.679823
std        42.298449
min         6.000000
25%       154.250000
50%       194.000000
75%       217.000000
max       251.000000
Name: text, dtype: float64


## Extract Concepts

If regenerate is set to True then the dataframes are regenerated and Both the dataframes are written in the csv format so we dont have to calculate them again. 

        dfne = dataframe of edges

        df = dataframe of chunks


Else the dataframes are read from the output directory

In [23]:
outputdirectory = 'graphs/'
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df, model='zephyr:latest')
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory + "graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory + "chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
print(dfg1.shape)
dfg1.head()

[
  {
    "node_1": "Jerry Adams",
    "node_2": "Patient ID: ",
    "edge": "Jerry Adams is the patient being discussed and reviewed in this context."
  },
  {
    "node_1": "Jerry Adams",
    "node_2": "69 y.o.",
    "edge": "Jerry Adams is a 69 year old male, as stated in the context."
  },
  {
    "node_1": "prostate cancer",
    "node_2": "started radiation therapy this week",
    "edge": "The patient has been diagnosed with prostate cancer and recently began radiation therapy for it, as mentioned in the context."
  },
  {
    "node_1": "Jerry Adams",
    "node_2": "compliant with his medications",
    "edge": "According to the context, Jerry Adams is adhering to his medication regimen for his underlying medical conditions."
  },
  {
    "node_1": "chest pain",
    "node_2": "shortness of breath",
    "edge": "The patient denies experiencing symptoms of chest pain and shortness of breath, as noted in the context."
  },
  {
    "node_1": "Jerry Adams",
    "node_2": "good supportiv

,node_1,node_2,edge,chunk_id,count
0,jerry adams,patient id:,Jerry Adams is the patient being discussed and...,38d3e5b0b5854945bd1ac139e7717c04,4
1,jerry adams,69 y.o.,"Jerry Adams is a 69 year old male, as stated i...",38d3e5b0b5854945bd1ac139e7717c04,4
2,prostate cancer,started radiation therapy this week,The patient has been diagnosed with prostate c...,38d3e5b0b5854945bd1ac139e7717c04,4
3,jerry adams,compliant with his medications,"According to the context, Jerry Adams is adher...",38d3e5b0b5854945bd1ac139e7717c04,4
4,chest pain,shortness of breath,The patient denies experiencing symptoms of ch...,38d3e5b0b5854945bd1ac139e7717c04,4


## Calculating contextual proximity

In [24]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
2649,trying to walk on daily basis,reviewed and updated as appropriate,"38d3e5b0b5854945bd1ac139e7717c04,38d3e5b0b5854...",6,contextual proximity
2653,vertigo,class 3 severe obesity due to excess calories ...,"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",2,contextual proximity
2654,vertigo,essential hypertension,"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",2,contextual proximity
2655,vertigo,malignant neoplasm of prostate (cms-hcc),"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",2,contextual proximity
2657,vertigo,mixed hyperlipidemia,"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",2,contextual proximity


### Merge both the dataframes

In [25]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg

,node_1,node_2,chunk_id,edge,count
0,69 y.o.,controlled,"52e8e4a1ea834e36b9af0905031fdae3,94975eda0cac4...",contextual proximity,3
1,69 y.o.,hyperlipidemia,"38d3e5b0b5854945bd1ac139e7717c04,52e8e4a1ea834...",contextual proximity,3
2,69 y.o.,hypertension,"38d3e5b0b5854945bd1ac139e7717c04,52e8e4a1ea834...",contextual proximity,3
3,69 y.o.,jerry adams,"38d3e5b0b5854945bd1ac139e7717c04,38d3e5b0b5854...",contextual proximity,13
4,69 y.o.,patient id:,"38d3e5b0b5854945bd1ac139e7717c04,94975eda0cac4...",contextual proximity,2
...,...,...,...,...,...
895,vertigo,class 3 severe obesity due to excess calories ...,"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",contextual proximity,2
896,vertigo,essential hypertension,"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",contextual proximity,2
897,vertigo,malignant neoplasm of prostate (cms-hcc),"6574280d6c5f42bab32af95103b63eb6,6574280d6c5f4...",contextual proximity,2
898,vertigo,meclizine (antivert),6574280d6c5f42bab32af95103b63eb6,"Causation relationship, as meclizine is prescr...",4


## Calculate the NetworkX Graph

In [26]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(115,)

In [27]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

### Calculate communities for coloring the nodes

In [28]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  4
[['69 y.o.', 'advised about diet', 'allergies', 'allergies, current medications, past family history, past medical history, past social history, past surgical history, problem list', 'appearance', 'atraumatic', 'chest pain', 'chronic medical problem', 'comfort eating', 'compliant with his medications', 'compliant with medications', 'conjunctiva/sclera', 'constitutional', 'controlled', 'current medications', 'eating more', 'follow up', 'good supportive system', 'has obesity', 'has vertigo', 'head', 'hyperlipidemia', 'hypertension', 'is a', 'is on radiation', 'is used for', 'jerry adams', 'light-headedness', 'negative', 'normal', 'normal appearance', 'obesity', 'past family history', 'past medical history', 'past social history', 'past surgical history', 'patient id:', 'patient id: ', 'prostate cancer', 'radiation', 'radiation therapy', 'recurrence', 'review and update', 'reviewed and updated as appropriate', 'rhinorrhea, sinus pain, sore throat', 'shortness of

### Create a dataframe for community colors

In [30]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,69 y.o.,#57d3db,1
1,advised about diet,#57d3db,1
2,allergies,#57d3db,1
3,"allergies, current medications, past family hi...",#57d3db,1
4,appearance,#57d3db,1
...,...,...,...
110,symptom,#91db57,3
111,time,#91db57,3
112,vertigo,#91db57,3
113,immunization due,#a157db,4


### Add colors to the graph

In [31]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [33]:
from pyvis.network import Network

graph_output_directory = "graphs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

graphs/index.html
